In [1]:
# Import necessary libraries
import numpy as np
import json
import requests
import pandas as pd
from geopy.distance import geodesic

/Users/dheeptharajesh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
df_resale = pd.read_csv('resale_data.csv')

print(df_resale.shape)
print(df_resale.info())
df_resale.head(2)

(155821, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   month                           155821 non-null  object 
 1   town                            155821 non-null  object 
 2   public_housing_flat_type        155821 non-null  object 
 3   block                           155821 non-null  object 
 4   street_name                     155821 non-null  object 
 5   storey_range                    155821 non-null  object 
 6   floor_area_sqm                  155821 non-null  float64
 7   flat_model                      155821 non-null  object 
 8   lease_commence_date             155821 non-null  int64  
 9   remaining_lease                 155821 non-null  object 
 10  resale_price_Singapore_dollars  155821 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.1+ MB
None


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0


In [6]:
def find_postal(lst, filename):
    
    for index, add in enumerate(lst):
        url = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal=' + add + '&returnGeom=Y&getAddrDetails=Y&pageNum=1'
        print(index, url)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Error in API response: {response.status_code}")
            continue

        try:
            data = json.loads(response.text)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue

        temp_df = pd.DataFrame.from_dict(data["results"])
        temp_df["address"] = add
    
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')



In [7]:
df_resale['Year'] = df_resale['month'].str[:4]
df_resale['mth'] = df_resale['month'].str[5:]
df_resale['address'] = df_resale['block'] + ' ' + df_resale['street_name']
display(df_resale)


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,01,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,01,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155816,2023-06,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,93 years 03 months,645000.0,2023,06,511B YISHUN ST 51
155817,2023-06,YISHUN,5 ROOM,513A,YISHUN ST 51,07 TO 09,120.0,3Gen,2018,93 years 09 months,685000.0,2023,06,513A YISHUN ST 51
155818,2023-06,YISHUN,5 ROOM,613,YISHUN ST 61,01 TO 03,121.0,Improved,1987,62 years 11 months,565000.0,2023,06,613 YISHUN ST 61
155819,2023-06,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,143.0,Apartment,1987,63 years 04 months,845000.0,2023,06,791 YISHUN AVE 2


In [8]:
def clean_address(string, dictionary):
    for item in string.split(" "):
        if item in dictionary.keys():
             string = string.replace(item,dictionary[item])
        else:
            string
    return string

In [10]:

hdb = pd.read_csv("hdb_address_geoloc.csv")

# remove repeated items in the geolocation csv file
hdb['block'] = hdb['address'].str.split().str.get(0)
hdb['true_add']= np.where(hdb['BLK_NO'] == hdb['block'], 'yes', 'no')
hdb1 =hdb[hdb['true_add']=='yes'] 
hdbfinal = hdb1.drop_duplicates(subset=['address'], keep='first')
hdbfinal.to_csv('hdb_address_geoloc_final.csv')
text_replace_dict = {"ST." : "ST" }
# Create new resale file with no missing data
df_resale_geopts = pd.read_csv('hdb_address_geoloc_final.csv')
df_resale_geopts = df_resale_geopts[['address', 'LATITUDE', 'LONGITUDE', 'POSTAL']]
df_resale["address_clean"] = [clean_address(val,text_replace_dict) for val in df_resale["address"]]
df_resale['address_clean'] = np.where(df_resale['address_clean'] == "4C ST GEORGE'S LANE", "4C ST. GEORGE'S LANE", 
                                      df_resale['address_clean'])



df_resale_postal= pd.merge(df_resale, df_resale_geopts, how='left', left_on=['address_clean'], right_on=['address'])
df_resale_postal.to_csv('resale-flat_withpostal.csv')

df_resale_postal.head(3)

,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address_x,address_clean,address_y,LATITUDE,LONGITUDE,POSTAL
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,1.362005,103.853880,560406.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,1.370966,103.838202,560108.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,1.380709,103.835368,560602.0


In [11]:
display(df_resale_postal)  

,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address_x,address_clean,address_y,LATITUDE,LONGITUDE,POSTAL
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,1.362005,103.853880,560406.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,1.370966,103.838202,560108.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,1.380709,103.835368,560602.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,01,465 ANG MO KIO AVE 10,465 ANG MO KIO AVE 10,465 ANG MO KIO AVE 10,1.366201,103.857201,560465.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,01,601 ANG MO KIO AVE 5,601 ANG MO KIO AVE 5,601 ANG MO KIO AVE 5,1.381041,103.835132,560601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155816,2023-06,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,93 years 03 months,645000.0,2023,06,511B YISHUN ST 51,511B YISHUN ST 51,511B YISHUN ST 51,1.415600,103.842414,762511.0
155817,2023-06,YISHUN,5 ROOM,513A,YISHUN ST 51,07 TO 09,120.0,3Gen,2018,93 years 09 months,685000.0,2023,06,513A YISHUN ST 51,513A YISHUN ST 51,513A YISHUN ST 51,1.416040,103.843489,761513.0
155818,2023-06,YISHUN,5 ROOM,613,YISHUN ST 61,01 TO 03,121.0,Improved,1987,62 years 11 months,565000.0,2023,06,613 YISHUN ST 61,613 YISHUN ST 61,613 YISHUN ST 61,1.420067,103.835284,760613.0
155819,2023-06,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,143.0,Apartment,1987,63 years 04 months,845000.0,2023,06,791 YISHUN AVE 2,791 YISHUN AVE 2,791 YISHUN AVE 2,1.420520,103.833564,760791.0


In [12]:
df_shoppingmall = pd.read_csv('shopping_mall_coordinates.csv')
display(df_shoppingmall)

,Mall Name,LATITUDE,LONGITUDE
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635
...,...,...,...
150,Gek Poh Shopping Centre,1.348742,103.697740
151,Rochester Mall,1.305408,103.788447
152,Taman Jurong Shopping Centre,1.334845,103.720462
153,West Coast Plaza,1.303586,103.766104


In [13]:
# hdb resale dataset with location
resale_postal = pd.read_csv("resale-flat_withpostal.csv")
resale_postal1 = resale_postal[['address_clean', 'LATITUDE', 'LONGITUDE']]
display(resale_postal1)

,address_clean,LATITUDE,LONGITUDE
0,406 ANG MO KIO AVE 10,1.362005,103.853880
1,108 ANG MO KIO AVE 4,1.370966,103.838202
2,602 ANG MO KIO AVE 5,1.380709,103.835368
3,465 ANG MO KIO AVE 10,1.366201,103.857201
4,601 ANG MO KIO AVE 5,1.381041,103.835132
...,...,...,...
155816,511B YISHUN ST 51,1.415600,103.842414
155817,513A YISHUN ST 51,1.416040,103.843489
155818,613 YISHUN ST 61,1.420067,103.835284
155819,791 YISHUN AVE 2,1.420520,103.833564


In [14]:
from geopy.distance import geodesic
import numpy as np

def find_nearest(house, amenity):
    results = {}

    for index, flat in enumerate(house.iloc[:, 0]):
        flat_loc = (house.iloc[index, 1], house.iloc[index, 2])

        flat_amenity = ['', '', 100]

        for ind, eachloc in enumerate(amenity.iloc[:, 0]):
            amenity_loc = (amenity.iloc[ind, 1], amenity.iloc[ind, 2])

            # Check for NaN values before calculating distance
            if np.isnan(flat_loc[0]) or np.isnan(flat_loc[1]) or np.isnan(amenity_loc[0]) or np.isnan(amenity_loc[1]):
                continue

            distance = geodesic(flat_loc, amenity_loc)

            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity

    return results


In [15]:
def save_file(data, filename):
    data1 = pd.DataFrame(data)
    data2 = data1.transpose()
    data2.to_csv(filename + '.csv', index=False)

In [16]:
# For Shopping Mall - # find nearest shopping mall to the flat
shoppingmall_postal = df_shoppingmall[['Mall Name', 'LATITUDE', 'LONGITUDE']]
display(shoppingmall_postal)     
shoppingmall_postal = find_nearest(resale_postal1 , shoppingmall_postal)
save_file(shoppingmall_postal,"shopping_mall_updated")



,Mall Name,LATITUDE,LONGITUDE
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635
...,...,...,...
150,Gek Poh Shopping Centre,1.348742,103.697740
151,Rochester Mall,1.305408,103.788447
152,Taman Jurong Shopping Centre,1.334845,103.720462
153,West Coast Plaza,1.303586,103.766104


In [17]:
# read hdb property info dataset
# HDB property information contains the location of existing HDB blocks, 
#highest floor level, year of completion, type of building and number of HDB flats (breakdown by flat type) per block etc.

df_property_info = pd.read_csv('hdb-property-information.csv')
df_property_info = df_property_info[['blk_no', 'street', 'max_floor_lvl', 'year_completed', 
                  'market_hawker', 'multistorey_carpark', 'total_dwelling_units']]

In [18]:
# update df_resale
df_resale = pd.read_csv('resale-flat_withpostal.csv')

In [20]:
# Prepare data for merging 
df_shopping_mall = pd.read_csv('shopping_mall_updated.csv')
# df_shopping_mall = df_shoppingmall.transpose()
# df_shopping_mall.reset_index(drop=True, inplace=True)
df_shopping_mall.columns = ['address','nearest_shopping_mall', 'distance_from_shopping_mall']
print(df_shopping_mall)

                    address          nearest_shopping_mall  \
0     406 ANG MO KIO AVE 10                        AMK Hub   
1      108 ANG MO KIO AVE 4                 Broadway Plaza   
2      602 ANG MO KIO AVE 5                 Broadway Plaza   
3     465 ANG MO KIO AVE 10  myVillage At Serangoon Garden   
4      601 ANG MO KIO AVE 5                 Broadway Plaza   
...                     ...                            ...   
9455    363B SEMBAWANG CRES                      Sun Plaza   
9456    365C SEMBAWANG CRES                      Sun Plaza   
9457    362A SEMBAWANG CRES                      Sun Plaza   
9458    366B SEMBAWANG CRES                      Sun Plaza   
9459    362B SEMBAWANG CRES                      Sun Plaza   

     distance_from_shopping_mall  
0           1.003018118556015 km  
1           0.868861431290418 km  
2          1.5255730377820635 km  
3          0.8937956343800417 km  
4          1.5693068531867669 km  
...                          ...  
9455      

In [21]:
# Merge datasets
resale_property_info = pd.merge(df_resale, df_property_info, how='left', 
               left_on=['block', 'street_name'], right_on=['blk_no', 'street'])
resale_and_shoppingmall = pd.merge(resale_property_info, df_shopping_mall, how='left', left_on=['address_clean'], right_on=['address'])

# clean after merge
merged_shoppingmall = resale_and_shoppingmall[:]

merged_shoppingmall['flat_remainlease'] =  merged_shoppingmall['remaining_lease'].str[:2].astype(int)
merged_shoppingmall['est_floor'] =  merged_shoppingmall['storey_range'].str[:2].astype(int)
merged_shoppingmall['distance_shopping_mall'] =  merged_shoppingmall['distance_from_shopping_mall'].str[:-2].astype(float)

merged_shoppingmall['shopping_mall_distribution'] =  np.where(merged_shoppingmall['distance_shopping_mall'] < 1, 'within-1km', 'More than-1km')

# export 
merged_shoppingmall.to_csv ("final_data_with_shopping_mall.csv",index = None, header=True)

df_final = pd.read_csv("final_data_with_shopping_mall.csv")
print(df_final.info())
df_final.head(3)

/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_1699/2813599204.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_shoppingmall['flat_remainlease'] =  merged_shoppingmall['remaining_lease'].str[:2].astype(int)
/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_1699/2813599204.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_shoppingmall['est_floor'] =  merged_shoppingmall['storey_range'].str[:2].astype(int)
/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_1699

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      155821 non-null  int64  
 1   month                           155821 non-null  object 
 2   town                            155821 non-null  object 
 3   public_housing_flat_type        155821 non-null  object 
 4   block                           155821 non-null  object 
 5   street_name                     155821 non-null  object 
 6   storey_range                    155821 non-null  object 
 7   floor_area_sqm                  155821 non-null  float64
 8   flat_model                      155821 non-null  object 
 9   lease_commence_date             155821 non-null  int64  
 10  remaining_lease                 155821 non-null  object 
 11  resale_price_Singapore_dollars  155821 non-null  float64
 12  Year            

,Unnamed: 0,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,market_hawker,multistorey_carpark,total_dwelling_units,address,nearest_shopping_mall,distance_from_shopping_mall,flat_remainlease,est_floor,distance_shopping_mall,shopping_mall_distribution
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,...,N,N,220,406 ANG MO KIO AVE 10,AMK Hub,1.003018118556015 km,61,10,1.003018,More than-1km
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,...,N,N,122,108 ANG MO KIO AVE 4,Broadway Plaza,0.868861431290418 km,60,1,0.868861,within-1km
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,...,N,N,150,602 ANG MO KIO AVE 5,Broadway Plaza,1.5255730377820635 km,62,1,1.525573,More than-1km
